In [99]:
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt

In [100]:
raw = pd.read_csv('botswana.tsv', sep='\t')
print raw.head()
%pylab inline

   ceb  age  educ    religion  idlnchld  knowmeth  usemeth  evermarr  agefm  \
0    0   18    10    catholic       4.0       1.0      1.0         0    NaN   
1    2   43    11  protestant       2.0       1.0      1.0         1   20.0   
2    0   49     4      spirit       4.0       1.0      0.0         1   22.0   
3    0   24    12       other       2.0       1.0      0.0         0    NaN   
4    3   32    13       other       3.0       1.0      1.0         1   24.0   

   heduc  urban  electric  radio   tv  bicycle  
0    NaN      1       1.0    1.0  1.0      1.0  
1   14.0      1       1.0    1.0  1.0      1.0  
2    1.0      1       1.0    1.0  0.0      0.0  
3    NaN      1       1.0    1.0  1.0      1.0  
4   12.0      1       1.0    1.0  1.0      1.0  
Populating the interactive namespace from numpy and matplotlib


## Постановка

Давайте проанализируем данные опроса 4361 женщин из Ботсваны:
botswana.tsv

О каждой из них мы знаем:

    сколько детей она родила (признак ceb)
    возраст (age)
    длительность получения образования (educ)
    религиозная принадлежность (religion)
    идеальное, по её мнению, количество детей в семье (idlnchld)
    была ли она когда-нибудь замужем (evermarr)
    возраст первого замужества (agefm)
    длительность получения образования мужем (heduc)
    знает ли она о методах контрацепции (knowmeth)
    использует ли она методы контрацепции (usemeth)
    живёт ли она в городе (urban)
    есть ли у неё электричество, радио, телевизор и велосипед (electric, radio, tv, bicycle)

Давайте научимся оценивать количество детей ceb по остальным признакам.

1.Загрузите данные и внимательно изучите их. Сколько разных значений принимает признак religion?

In [101]:
raw.describe()

,ceb,age,educ,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
count,4361.000000,4361.000000,4361.000000,4241.000000,4354.000000,4290.000000,4361.000000,2079.000000,1956.000000,4361.000000,4358.000000,4359.000000,4359.000000,4358.000000
mean,2.441642,27.405182,5.855996,4.615892,0.963252,0.577622,0.476726,20.686388,5.144683,0.516625,0.140202,0.701766,0.092911,0.275815
std,2.406861,8.685233,3.927075,2.219303,0.188164,0.493996,0.499515,5.002383,4.803028,0.499781,0.347236,0.457535,0.290341,0.446975
min,0.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,20.000000,3.000000,3.000000,1.000000,0.000000,0.000000,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,26.000000,7.000000,4.000000,1.000000,1.000000,0.000000,20.000000,6.000000,1.000000,0.000000,1.000000,0.000000,0.000000
75%,4.000000,33.000000,8.000000,6.000000,1.000000,1.000000,1.000000,23.000000,8.000000,1.000000,0.000000,1.000000,0.000000,1.000000
max,13.000000,49.000000,20.000000,20.000000,1.000000,1.000000,1.000000,46.000000,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [102]:
print raw.religion.value_counts()
print '\nAnswer 1: ', len(raw.religion.value_counts())

spirit        1841
other         1080
protestant     993
catholic       447
Name: religion, dtype: int64

Answer 1:  4


In [103]:
#Оценим сбалансированность выборки по категориальным признакам:
print raw.religion.value_counts()

print raw.knowmeth.value_counts()
print raw.usemeth.value_counts()
print raw.evermarr.value_counts()
print raw.urban.value_counts()
print raw.electric.value_counts()
print raw.radio.value_counts()
print raw.tv.value_counts()
print raw.bicycle.value_counts()

spirit        1841
other         1080
protestant     993
catholic       447
Name: religion, dtype: int64
1.0    4194
0.0     160
Name: knowmeth, dtype: int64
1.0    2478
0.0    1812
Name: usemeth, dtype: int64
0    2282
1    2079
Name: evermarr, dtype: int64
1    2253
0    2108
Name: urban, dtype: int64
0.0    3747
1.0     611
Name: electric, dtype: int64
1.0    3059
0.0    1300
Name: radio, dtype: int64
0.0    3954
1.0     405
Name: tv, dtype: int64
0.0    3156
1.0    1202
Name: bicycle, dtype: int64


2.Во многих признаках есть пропущенные значения. Сколько объектов из 4361 останется, если выбросить все, содержащие пропуски? 

In [104]:
data = raw.dropna()

print '\nAnswer 2: ', data.count()[0]


Answer 2:  1834


В разных признаках пропуски возникают по разным причинам и должны обрабатываться по-разному.

Например, в признаке agefm пропуски стоят только там, где evermarr=0, то есть, они соответствуют женщинам, никогда не выходившим замуж. Таким образом, для этого признака NaN соответствует значению "не применимо".

В подобных случаях, когда признак x1 на части объектов в принципе не может принимать никакие значения, рекомендуется поступать так:

    создать новый бинарный признак
    x2=1,x1='не применимо',0,иначе;
    заменить "не применимо" в x1 на произвольную константу c, которая среди других значений x1 не встречается.

Теперь, когда мы построим регрессию на оба признака и получим модель вида
y=β0+β1x1+β2x2,
на тех объектах, где x1 было измерено, регрессионное уравнение примет вид
y=β0+β1x,
а там, где x1 было "не применимо", получится
y=β0+β1c+β2.
Выбор c влияет только на значение и интерпретацию β2, но не β1.

Давайте используем этот метод для обработки пропусков в agefm и heduc.

    Создайте признак nevermarr, равный единице там, где в agefm пропуски.
    Удалите признак evermarr — в сумме с nevermarr он даёт константу, значит, в нашей матрице X будет мультиколлинеарность.
    Замените NaN в признаке agefm на cagefm=0.
    У объектов, где nevermarr = 1, замените NaN в признаке heduc на cheduc1=−1 (ноль использовать нельзя, так как он уже встречается у некоторых объектов выборки).

3.Сколько осталось пропущенных значений в признаке heduc?

In [105]:
data = raw.copy()
data['nevermarr'] = data['agefm'].apply(lambda x: 1 if np.isnan(x) else 0)
data.drop('evermarr', axis=1, inplace=True)
c_agefm = 0
c_heduc1 = -1
data['agefm'].fillna(value=c_agefm, inplace=True)
data.ix[data['nevermarr'] == 1, 'heduc'] = c_heduc1
data.describe()

,ceb,age,educ,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
count,4361.000000,4361.000000,4361.000000,4241.000000,4354.000000,4290.000000,4361.000000,4238.000000,4361.000000,4358.000000,4359.000000,4359.000000,4358.000000,4361.000000
mean,2.441642,27.405182,5.855996,4.615892,0.963252,0.577622,9.861729,1.836008,0.516625,0.140202,0.701766,0.092911,0.275815,0.523274
std,2.406861,8.685233,3.927075,2.219303,0.188164,0.493996,10.894991,4.475487,0.499781,0.347236,0.457535,0.290341,0.446975,0.499515
min,0.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,20.000000,3.000000,3.000000,1.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,26.000000,7.000000,4.000000,1.000000,1.000000,0.000000,-1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000
75%,4.000000,33.000000,8.000000,6.000000,1.000000,1.000000,19.000000,4.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000
max,13.000000,49.000000,20.000000,20.000000,1.000000,1.000000,46.000000,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [106]:
print 'Answer 3: ', data.age.count() - data.heduc.count()

Answer 3:  123


Избавимся от оставшихся пропусков.

Для признаков idlnchld, heduc и usemeth проведите операцию, аналогичную предыдущей: создайте индикаторы пропусков по этим признакам (idlnchld_noans, heduc_noans, usemeth_noans), замените пропуски на нехарактерные значения (cidlnchld=−1, cheduc2=−2 (значение -1 мы уже использовали), cusemeth=−1).

Остались только пропуски в признаках knowmeth, electric, radio, tv и bicycle. Их очень мало, так что удалите объекты, на которых их значения пропущены.

4.Какого размера теперь наша матрица данных? Умножьте количество строк на количество всех столбцов (включая отклик ceb).

In [113]:
data['idlnchld_noans'] = data['idlnchld'].apply(lambda x: 1 if np.isnan(x) else 0)
data['heduc_noans'] = data['heduc'].apply(lambda x: 1 if np.isnan(x) else 0)
data['usemeth_noans'] = data['usemeth'].apply(lambda x: 1 if np.isnan(x) else 0)
c_idlnchld = -1
c_heduc2 = -1
c_usemeth = -1
data['idlnchld'].fillna(value=c_idlnchld, inplace=True)
data['heduc'].fillna(value=c_heduc2, inplace=True)
data['usemeth'].fillna(value=c_usemeth, inplace=True)
data.dropna(inplace=True)
data.describe()

,ceb,age,educ,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr,idlnchld_noans,heduc_noans,usemeth_noans
count,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000
mean,2.437443,27.393514,5.863155,4.466191,0.963431,0.553818,9.850736,1.758510,0.517249,0.140064,0.701702,0.092916,0.275299,0.523919,0.027139,0.028059,0.015179
std,2.401785,8.675630,3.922694,2.372879,0.187722,0.526808,10.897246,4.439519,0.499760,0.347094,0.457564,0.290348,0.446716,0.499485,0.162507,0.165160,0.122280
min,0.000000,15.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,20.000000,3.000000,3.000000,1.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,26.000000,7.000000,4.000000,1.000000,1.000000,0.000000,-1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,4.000000,33.000000,8.000000,6.000000,1.000000,1.000000,19.000000,4.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
max,13.000000,49.000000,20.000000,20.000000,1.000000,1.000000,46.000000,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [93]:
print 'Answer 4: ', data.age.count()*len(data.columns)

Answer 4:  78264


5.Постройте регрессию количества детей ceb на все имеющиеся признаки методом smf.ols, как в разобранном до этого примере. Какой получился коэффициент детерминации R2? Округлите до трёх знаков после десятичной точки.

### Простейшая модель

In [95]:
data.columns

Index([u'ceb', u'age', u'educ', u'religion', u'idlnchld', u'knowmeth',
       u'usemeth', u'agefm', u'heduc', u'urban', u'electric', u'radio', u'tv',
       u'bicycle', u'nevermarr', u'idlnchld_noans', u'heduc_noans',
       u'usemeth_noans'],
      dtype='object')

In [115]:
# Построим линейную модель по всем признакам.
m1 = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + \
                    radio + tv + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=data)
fitted = m1.fit()
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Tue, 11 Apr 2017   Prob (F-statistic):               0.00
Time:                        18:55:38   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------
Intercept                 -1

In [116]:
print 'Answer 5: ', round(fitted.rsquared,3)

Answer 5:  0.644


In [117]:
# Используем критерий Бройша-Пагана для проверки гомоскедастичности ошибок:
print 'Breusch-Pagan test: p=%f' % sms.het_breushpagan(fitted.resid, fitted.model.exog)[1]

Breusch-Pagan test: p=0.000000


Ошибки гетероскедастичны, значит, значимость признаков может определяться неверно. Сделаем поправку Уайта.

Удалите из модели незначимые признаки religion, radio и tv. Проверьте гомоскедастичность ошибки, при необходимости сделайте поправку Уайта.

Не произошло ли значимого ухудшения модели после удаления этой группы признаков? Проверьте с помощью критерия Фишера. Чему равен его достигаемый уровень значимости? Округлите до четырёх цифр после десятичной точки.

Если достигаемый уровень значимости получился маленький, верните все удалённые признаки; если он достаточно велик, оставьте модель без религии, тв и радио.

In [121]:
m2 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + \
                    bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=data)
fitted = m2.fit()
print 'Breusch-Pagan test: p=%f' % sms.het_breushpagan(fitted.resid, fitted.model.exog)[1]
# Ошибки гетероскедастичны, значит, значимость признаков может определяться неверно. Сделаем поправку Уайта.
fitted = m2.fit(cov_type='HC1')
print fitted.summary()

Breusch-Pagan test: p=0.000000
                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Tue, 11 Apr 2017   Prob (F-statistic):               0.00
Time:                        19:04:21   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept    

In [124]:
(F, p, k1) = m1.fit().compare_f_test(m2.fit())
print "F=%f, p=%f, k1=%f" % (F, p, k1)

F=0.919236, p=0.467231, k1=5.000000


Модель не стала хуже.

In [125]:
print 'Answer 8: ', round(p,4)

Answer 8:  0.4672


In [126]:
m3 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + agefm + heduc + urban + electric + \
                    bicycle + nevermarr + idlnchld_noans + heduc_noans', 
             data=data)
fitted = m3.fit()
print 'Breusch-Pagan test: p=%f' % sms.het_breushpagan(fitted.resid, fitted.model.exog)[1]
# Ошибки гетероскедастичны, значит, значимость признаков может определяться неверно. Сделаем поправку Уайта.
fitted = m3.fit(cov_type='HC1')
print fitted.summary()

Breusch-Pagan test: p=0.000000
                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     396.4
Date:                Tue, 11 Apr 2017   Prob (F-statistic):               0.00
Time:                        19:10:29   Log-Likelihood:                -7825.7
No. Observations:                4348   AIC:                         1.568e+04
Df Residuals:                    4335   BIC:                         1.576e+04
Df Model:                          12                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept    

In [134]:
(F, p, k1) = m2.fit().compare_f_test(m3.fit())
print "F=%f, p=%g, k1=%f" % (F, p, k1)
s = str(p)
print 'Answer 9:', abs(int(s[s.find('e') + 1:len(s)]))

F=92.890582, p=3.1552e-40, k1=2.000000
Answer 9: 40


In [135]:
b1 = 0.0770
b2 = 0.6565
y = b1*c_idlnchld + b2
print y

0.5795
